In [12]:
import os
import torch
import random
import evaluate
import transformers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional
from dataclasses import dataclass 
from time import perf_counter
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, disable_progress_bar
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    EarlyStoppingCallback
)

In [3]:
class finetuned_model():
    

    def __init__(self):
        self.fine_tuned_model_checkpoint = './training/data/english_to_spanish/mt5-small_en-sp/checkpoint-4500'
        self.tokenizer = AutoTokenizer.from_pretrained(self.fine_tuned_model_checkpoint)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(self.fine_tuned_model_checkpoint)
        print('Model loaded')

c:\Users\Jorge\anaconda3\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


model loaded


In [4]:
def generate_translation2(model, tokenizer, example):
    """print out the source, target and predicted raw text."""
    source = example
    print(source)
    #target = example[config.target_lang]
    #print(target)
    input_ids = tokenizer(source)["input_ids"]
    input_ids = torch.LongTensor(input_ids).view(1, -1).to(model.device)
    print(input_ids)
    #input_ids=input_ids[0]
    generated_ids = model.generate(inputs=input_ids, max_length=20)
    prediction = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    print('source: ', source)
    #print('target: ', target)
    print('prediction: ', prediction)

In [5]:
flan_model=finetuned_model()

In [6]:
flan_model.model

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [7]:
flan_model.tokenizer

T5TokenizerFast(name_or_path='./training/data/english_to_spanish/mt5-small_en-sp/checkpoint-4500', vocab_size=250100, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [8]:
text='I enjoy my days with you'

In [10]:
generate_translation2(flan_model.model, flan_model.tokenizer, text)

I enjoy my days with you
tensor([[ 336, 9070, 1037, 5382,  514,  521,    1]])
source:  I enjoy my days with you
prediction:  Me encanta mis días contigo.


In [11]:
text='I like to eat apples and oranges'
generate_translation2(flan_model.model, flan_model.tokenizer, text)

I like to eat apples and oranges
tensor([[  336,  1469,   288, 19225, 44790,   263,   305,  2855,   299,     1]])
source:  I like to eat apples and oranges
prediction:  Me gusta comer comida de manzanas y plantas.
